In [ ]:
# 파이썬으로 이메일 발송하기

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import ImageFont, ImageDraw, Image
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import time

# 이메일 발송 함수
def send_email(last_email_time):
    current_time = time.time()
    if current_time - last_email_time < 300:  # 5분 = 300초
        return last_email_time, False

    gmail_user = 'email'
    gmail_password = 'password'
    sent_from = gmail_user
    to = ['email']
    subject = '이상행동 감지'
    body = '이상행동이 감지됐습니다.'

    msg = MIMEMultipart()
    msg['From'] = sent_from
    msg['To'] = ", ".join(to)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain', 'utf-8'))

    try:
        smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        smtp_server.ehlo()
        smtp_server.login(gmail_user, gmail_password)
        smtp_server.sendmail(sent_from, to, msg.as_string())
        smtp_server.close()
        print("Email sent successfully!")
        return current_time, True
    except Exception as ex:
        print("Something went wrong...", ex)
        return last_email_time, False

# 테스트 영상 파일 경로
test_video_path = "C:/Users/user/Downloads/20240103_184644.mp4"

# 모델 불러오기
model_path = "D:/Models/CNN_LSTM_9.h5"
model = load_model(model_path)

class_names = {0: '파손', 1: '구매', 2: '폭행', 3: '절도'}

# 한글 폰트 경로 지정 (시스템에 설치된 한글 폰트 경로를 사용)
font_path = "C:/Windows/Fonts/malgun.ttf"
font_size = 30
font = ImageFont.truetype(font_path, font_size)

# VideoCapture를 사용하여 테스트 영상 열기
cap = cv2.VideoCapture(test_video_path)

# VideoCapture를 사용하여 테스트 영상의 속성 가져오기
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# VideoWriter 객체 생성
output_path = "D:/test_5.mp4"  # 결과를 저장할 파일 경로
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


last_email_time = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    resized_frame = cv2.resize(frame, (100, 100))
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    normalized_frame = gray_frame / 255.0
    frame_buffer.append(normalized_frame.flatten())  # 특징 추출 후 버퍼에 저장
    if len(frame_buffer) == 50:  # 50프레임씩 모아서 처리
        input_data = np.array(frame_buffer)  # 50프레임을 NumPy 배열로 변환
        input_data = input_data.reshape(1, 50, 100, 100, 1)  # 모델에 입력하기 위해 형태 변환
        predictions = model.predict(input_data)  # 모델로 예측 수행
        predicted_class = class_names[np.argmax(predictions[0])]  # 예측된 클래스 이름
        predictions_buffer.append(predicted_class)  # 예측된 클래스 이름 저장
        frame_buffer = []  # 버퍼 비우기
    # PIL 라이브러리를 사용하여 한글 텍스트 추가
    if predictions_buffer:
        text = f"행동: {predictions_buffer[-1]}"
        frame_pil = Image.fromarray(frame)
        draw = ImageDraw.Draw(frame_pil)
        draw.text((50, 50), text, font=font, fill=(0, 0, 255))
        frame = np.array(frame_pil)

    # 오버레이된 프레임을 저장
    out.write(frame)

    # 이메일 조건 확인 및 발송
    if predictions_buffer and predictions_buffer[-1] in ['파손', '폭행', '절도']:
        last_email_time, email_sent = send_email(last_email_time)
        if email_sent:
            predictions_buffer = []  # 이메일 발송 후 버퍼 초기화


# 작업 완료 후 해제
cap.release()
out.release()
cv2.destroyAllWindows()
